## Setup, Installations, and Imports

In [ ]:
#install packages as necessary
!pip -q install accelerate datasets transformers evaluate jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 6.6 MB/s eta 0:00:00


In [ ]:
#import our packages
import os
import torch
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
from datasets import load_metric
from transformers import AutoImageProcessor, AutoModelForImageClassification, Trainer, TrainingArguments, BlipProcessor, BlipForConditionalGeneration
import matplotlib.pyplot as plt
import accelerate
import numpy as np
import pickle
import random
import tqdm
import requests
import evaluate
from PIL import Image
import copy
from rich.progress import track

In [ ]:
#mount the drive (for saving files to google drive (so the data will not have to be reprocessed everytime colab restarts a session))
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Poliphonic Data Extraction and Preprocessing

In [ ]:
'''
Note: 
- This section is just meant to extract the data from the original tgz file, process it into a Poliphonic dataset object, and save it accordingly (such that it can persist beyond individual Google Colab sessions)
If all of the above has already been done, skip this section.
- This section was originally a part of code for another model we were experimenting with. Not all of it is necessary for the finetuned blip model but since
the datasets were already created anyway for prior experiments, we leave it and leverage a piece of it for the finetuned blip model
'''

In [ ]:
#this specific cell block is based on the original paper that created the raw dataset we're using.
#it's been modified/adapted for our use case but credit goes to that paper and its authors. It provides some processing of bekrn files
#that has been helpful for our group, since we are not familiar with the notation or the predecessor notation it was based on

def load_data(partition_file, resize_ratio=1., use_raw_krn=False, load_distorted=False, extension=".bekrn"):
    X = []
    Y = []
    with open(partition_file) as partfile:
        part_lines = partfile.read()
        part_lines = part_lines.split("\n")
        # print(part_lines)
        for file_path in track(part_lines, description="Loading..."):
            if extension != ".bekrn":
                file_path = file_path.replace(".bekrn", extension)
            krn = None
            krnlines = []
            file_path = f"{file_path}"
            # print(file_path)
            if os.path.isfile(file_path):
                with open(file_path) as krnfile:
                    try:
                      krn = krnfile.read()
                    except:
                      continue
                    krn = krn.replace(" ", " <s> ")
                    krn = krn.replace("·", " ")
                    lines = krn.split("\n")
                    for line in lines:
                        line = line.replace("\t", " <t> ")
                        line = line.split(" ")
                        if len(line) > 1:
                            line.append("<b>")
                            krnlines.append(line)
                    # print(os.path.exists(f"{file_path.split('.')[0]}.jpg"))
                    if os.path.exists(f"{file_path.split('.')[0]}.jpg"):
                        # print("HELLO")
                        if load_distorted:
                            height = 256
                            img = cv2.imread(f"{file_path.split('.')[0]}_distorted.jpg", 0)
                            width = int(float(height * img.shape[1]) / img.shape[0])
                            img =  cv2.resize(img, (width, height), interpolation=cv2.INTER_LINEAR)
                            if (height//8) * (width//16) > len(sum(krnlines, [])):
                                width = int(np.ceil(img.shape[1] * resize_ratio))
                                height = int(np.ceil(img.shape[0] * resize_ratio))
                                img = cv2.resize(img, (width, height), interpolation=cv2.INTER_LINEAR)
                                img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
                                X.append(img)
                                Y.append(sum(krnlines, []))
                        else:
                            img = cv2.imread(f"{file_path.split('.')[0]}.jpg", 0)
                            width = int(np.ceil(img.shape[1] * resize_ratio))
                            height = int(np.ceil(img.shape[0] * resize_ratio))
                            img = cv2.resize(img, (width, height), interpolation=cv2.INTER_LINEAR)
                            img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
                            X.append(img)
                            Y.append(sum(krnlines, []))
            # print(len(X))
            # print(len(krnlines))
    print(len(X)), print(len(Y))
    return X, Y


class PoliphonicDataset(Dataset):
    def __init__(self, partition_file) -> None:
        self.x, self.y = load_data(partition_file)
        self.tensorTransform = transforms.ToTensor()

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        image = self.tensorTransform(self.x[index])
        gt = torch.from_numpy(np.asarray([self.w2i[token] for token in self.y[index]]))

        return image, gt, (image.shape[2] // 8) * (image.shape[1] // 16), len(gt)

    def get_max_hw(self):
        m_width = np.max([img.shape[1] for img in self.x])
        m_height = np.max([img.shape[0] for img in self.x])

        return m_height, m_width

    def get_max_seqlen(self):
        return np.max([len(seq) for seq in self.y])

    def vocab_size(self):
        return len(self.w2i)

    def get_gt(self):
        return self.y

    def set_dictionaries(self, w2i, i2w):
        self.w2i = w2i
        self.i2w = i2w
        self.padding_token = w2i['<pad>']

    def get_dictionaries(self):
        return self.w2i, self.i2w

    def get_i2w(self):
        return self.i2w

In [ ]:
#unzip the tgz raw data file.
#note: change the below string paths as appropriate.

!tar -xzf "grandstaff.tgz" -C "grandstaff_dataset/"

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extende

In [ ]:
#Set up the train, val, and test sets as Dataset objects
train_path = "drive/MyDrive/train_shuffled.txt"
val_path = "drive/MyDrive/val_shuffled.txt"
test_path = "drive/MyDrive/test_shuffled.txt"

#create the PoliphonicDataset objects
train_dataset = PoliphonicDataset(partition_file=train_path)
val_dataset = PoliphonicDataset(partition_file=val_path)
test_dataset = PoliphonicDataset(partition_file=test_path)


In [ ]:
#double check that the lengths of the datasets we just read in are as expected
len(train_dataset), len(test_dataset), len(val_dataset)

(32330, 10776, 10776)

### Comments

All looks good so far, not much to say as most code came from the original paper

## Create the train, test, and val files

In [ ]:
'''
Note: This section is only for denominating which samples will go into train vs test vs val datasets. If that's already been determined by some other method,
skip this section.
'''

In [ ]:
#let's get the total number of samples from the dataset first
sample_bekrn = [] #here, we're only interested in the bekern files (vs the krn files)

for root, dirs, files in os.walk("grandstaff_dataset/"):
   for name in files:
      if 'bekrn' in name:
        print(os.path.join(root, name))
        # Might consider removing the print statement if not needed for testing
        sample_bekrn.append(os.path.join(root, name))

Streaming output truncated to the last 5000 lines.
grandstaff_dataset/chopin/mazurkas/mazurka33-2/min3_up_m-16-20.bekrn
grandstaff_dataset/chopin/mazurkas/mazurka33-2/maj2_down_m-8-12.bekrn
grandstaff_dataset/chopin/mazurkas/mazurka33-2/min3_down_m-84-87.bekrn
grandstaff_dataset/chopin/mazurkas/mazurka33-2/min3_down_m-21-24.bekrn
grandstaff_dataset/chopin/mazurkas/mazurka33-2/maj2_up_m-18-21.bekrn
grandstaff_dataset/chopin/mazurkas/mazurka33-2/maj3_down_m-85-90.bekrn
grandstaff_dataset/chopin/mazurkas/mazurka33-2/min3_down_m-126-129.bekrn
grandstaff_dataset/chopin/mazurkas/mazurka33-2/maj2_down_m-116-120.bekrn
grandstaff_dataset/chopin/mazurkas/mazurka33-2/min3_down_m-96-99.bekrn
grandstaff_dataset/chopin/mazurkas/mazurka33-2/maj3_up_m-128-132.bekrn
grandstaff_dataset/chopin/mazurkas/mazurka33-2/maj2_up_m-54-57.bekrn
grandstaff_dataset/chopin/mazurkas/mazurka33-2/min3_down_m-30-33.bekrn
grandstaff_dataset/chopin/mazurkas/mazurka33-2/original_m-128-132.bekrn
grandstaff_dataset/chopin/ma

In [ ]:
len(sample_bekrn)

In [ ]:
#let's split the train-val-test sets such that each are 60%, 20%, and 20% of all the 53889 samples we have
train_pt, val_pt = 53889*0.6, 53889*0.8
train_pt, val_pt

(32333.399999999998, 43111.200000000004)

### Code Comments

Might have been better to use len(sample_bekrn) so this can be applied to other datasets. Avoiding hardcoded numbers is always preferable

In [ ]:
#do the split and double check that it works as expected

#shuffle the list beforehand just in case so there isn't any weird slant in the data between train vs. val vs. test (since it is possible that
#beethoven snippets or diff from, say, scarletti)
random.seed(0) #seed for reproducibility
random.shuffle(sample_bekrn)

#split the data into different sets
train, val, test = sample_bekrn[:32333], sample_bekrn[32333:43111], sample_bekrn[43111:]

#and double check the size
len(train), len(val), len(test)

(32333, 10778, 10778)

### Code Comments

Again using the hardcoded values rather than train_pt, val_pt

Also wonder if the sklearn train_test_split works on PT tensors

In [ ]:
#save the different dataset records to text files
def write_testset_files(data, file_name):
  '''
  Save the given data in a file determined by the file_name. Samples are separated by a new line.
  '''
  with open(f'{file_name}', 'w') as f:
      for samp in data:
          f.write(f"{samp}\n")

In [ ]:
#save it for use later just in case
write_testset_files(train, 'drive/MyDrive/train_shuffled.txt')
write_testset_files(test, 'drive/MyDrive/test_shuffled.txt')
write_testset_files(val, 'drive/MyDrive/val_shuffled.txt')

## How do multimodals do out of the box?

In [ ]:
### let's test a popular vision-language model from salesforce called blip.
### It seems to mostly be used for tasks like image-captioning, but maybe we can train it to treat the sequence of musical notes as the 'caption'?

In [ ]:
#this cell block is from huggingface documentation for BLIP
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

img_url = 'grandstaff_dataset/beethoven/piano-sonatas/sonata31-2/maj2_down_m-1-5.jpg' #a random jpg image from our data
raw_image = Image.open(img_url).convert('RGB')

# conditional image captioning
text = "a list of music symbols starting with the symbol for " #i tested a number of different leading texts here but none worked well
inputs = processor(raw_image, text, return_tensors="pt").to("cuda", torch.float16)

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

# unconditional image captioning
inputs = processor([raw_image, raw_image], return_tensors="pt").to("cuda", torch.float16) #sent the raw_image in twice just to sanity check that batched outputs are in the shape and have the content I expect

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))
print(out.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a list of music symbols starting with the symbol for the letter f
a black and white image of a musical score with notes


### Code Comments

This is a very cool demonstration of exactly why you might want to finetune a model.
It performs properly but just cannot provide a specific enough output.

In [ ]:
## the above is not really unreasonable but it's not useful nor specific enough. let's try finetuning to see if the model can do better?

## Finetuning BLIP

In [ ]:
torch.cuda.empty_cache() #run to help free up a bit of memory if i'm out of it

In [ ]:
#ok, let's try finetuning and see if we can get better results that way
class blip_music_dataset(torch.utils.data.Dataset):
    '''
    Make a music dataset object that can be then used as input for the BLIP model.
    '''
    def __init__(self, data_path, processor, poliphonic_formatted_gt, transforms=None):
        '''
        Constructor class.
        Args:
          data_path (str): the path to the data for which we want to create a Dataset object
          processor (obj): the img-text processor for inputs that are intended for blip
          poliphonic_formatted_gt (list): the list of processed ground truth text labels (processed by the Poliphonic dataset beforehand)
          transforms (dict): optional image transformations applied before the image is sent to the processor and blip
        '''
        #set up the inputs
        self.data_path=data_path
        self.processor=processor
        self.gt = poliphonic_formatted_gt
        if transforms:
          self.transforms = transforms
        else:
          self.transforms = None

        #read in the sample file paths in the data file
        self.sample_paths = []
        with open(self.data_path, 'r') as f:
          for line in f:
            self.sample_paths.append(line)

        #run a sweep to make sure there are no paths that coudl cause errors later
        self.check_exists()

    def check_exists(self):
      '''
      With this particular grandstaff_dataset source, there appear to be a few missing image files (i.e. bekrn exists but no jpg.).
      This function roots them out so as not to cause errors when __getitem__ is called.
      '''
      with open(self.data_path) as f:
        for line in f:
          new = line.split('.')[0]+'.jpg'
          if os.path.exists(new) == False:
            print(f"Could not locate image file for: {new}")
            self.sample_paths.remove(line)

    def __len__(self):
      '''
      Get the number of samples in the dataset.

      Returns:
        int: the length/size of the dataset
      '''
      return len(self.sample_paths)

    def __getitem__(self, idx):
      '''
      Retrieve processed sample from dataset

      Args:
        idx (int): the index of the sample within the dataset
      Returns:
        encoding (dict): dictionary of information about the dataset, including pixel_values, labels, attention_mask
      '''

      #get the image pixels
      img_path = self.sample_paths[idx].split('.')[0] + '.jpg'
      # print(img_path)
      raw_image = Image.open(img_path).convert('RGB')
      if self.transforms:
        raw_image = self.transforms(raw_image)

      #labels
      labels = "".join(self.gt[idx])

      #send it to the blip processor (due to model + gpu constraints, I'm going to allow for truncation)
      encoding = self.processor(images=raw_image, text=labels, max_length=None, padding='max_length', truncation=True, return_tensors="pt")

      # remove batch dimension (this was taken from huggingface documentation)
      encoding = {k:v.squeeze() for k,v in encoding.items()}

      return encoding

### Code Comments
I actually wrote a python package for reading and writing filelists like you in the constructor. Checkout filelister on pip if you're ever curious.

Dataset looks good. I wonder if saving preprocessed images (convert to rgb, transform, etc.) instead of preprocessing in getitem would be faster in the long run.

In [ ]:
#make datasets and dataloaders. I kept the batch size=2 because I kept running out of cuda memory later on in the training loop if batch size > 2
train_dataset_blip = blip_music_dataset('drive/MyDrive/train_shuffled.txt', processor, train_dataset.get_gt(), transforms=None)
train_dataloader_blip = DataLoader(train_dataset_blip, shuffle=True, batch_size=2)

val_dataset_blip = blip_music_dataset('drive/MyDrive/val_shuffled.txt', processor, val_dataset.get_gt(), transforms=None)
val_dataloader_blip = DataLoader(val_dataset_blip, shuffle=True, batch_size=2)

test_dataset_blip = blip_music_dataset('drive/MyDrive/test_shuffled.txt', processor, test_dataset.get_gt(), transforms=None)
test_dataloader_blip = DataLoader(test_dataset_blip, shuffle=True, batch_size=2)


Could not locate image file for: grandstaff_dataset/mozart/piano-sonatas/sonata15-2/.jpg
Could not locate image file for: grandstaff_dataset/mozart/piano-sonatas/sonata03-1/.jpg
Could not locate image file for: grandstaff_dataset/hummel/preludes/prelude67-01/.jpg
Could not locate image file for: grandstaff_dataset/hummel/preludes/prelude67-01/.jpg
Could not locate image file for: grandstaff_dataset/chopin/mazurkas/mazurka30-4/.jpg
Could not locate image file for: grandstaff_dataset/mozart/piano-sonatas/sonata06-3e/.jpg
Could not locate image file for: grandstaff_dataset/chopin/mazurkas/mazurka41-2/.jpg


In [ ]:
#note: it looks like from above that the extra '/' before '.jpg' is causing the path error but even after removing those, I could not find any associated image files
#therefore, I'm just removing them since there aren't a lot (7 bekrn paths that caused errors)
#as an example:
os.path.exists("grandstaff_dataset/chopin/mazurkas/mazurka41-2.jpg")

False

In [ ]:
#double check what the format of a sample looks like + that it looks as expected, esp. if it was just read in in this session
test = next(iter(train_dataloader_blip))['input_ids']
test

tensor([[ 101, 1008, 1008,  ..., 1028, 1012,  102],
        [ 101, 1008, 1008,  ..., 2497, 1026,  102]])

### Code comments

Again, I understand the use of lines like this, but might want to go through and clean up extra print statements, etc. before submission

In [ ]:
#now let's load in the metric we want to use later: Word Error Rate.

#This (roughly) measures the number of "errors" -- where errors is defined as differences between what was predicted and the ground truth reference --
#(e.g., deletions, substitutions, insertions, etc.) divided by the number of words in the reference. The higher the WER value, the worse the match; WER = 0 indicates a perfect generation/prediction.

#We picked this metric over others for many reasons. It is sensitive to exact matches of words as well as the order of the words (which rouge-n, another popular image-captioning metric, does not always appear to be),
#and allows for more reasonable flexibility than other metrics like f1/precision/recall for our use case.

wer = evaluate.load('wer')

### Code Comments

You might prefer moving some longer comments like this to docstrings or separate markdown blocks

In [1]:
#note: to avoid unfairly penalizing the model during WER eval, I put in a section below to truncate the ground truth label to 500 tokens if need be
#for the purposes of wer evaluation only. 

In [ ]:
#setting up our optimizer with 5e-5 learning rate
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

#prep the datasets that will be used/seen during the training loop
name_datasets = ['train', 'val']
datasets = [train_dataloader_blip, val_dataloader_blip]

#set the device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

#initialize the metrics
best_wer = 1.0 #wer = 1 is the worst possible score; wer = 0 is the best possible score

#let's run a simple training loop (not putting this in a function since I'm not really running this for a bunch of different datasets)
for epoch in range(2): #keeping the epoch numbers small because performance seemed far worse when we previously trained beyond a few epochs

  print("Epoch:", epoch)

  for name, data in zip(name_datasets, datasets):

    if name == 'train':
      model.train() #set model to training mode
    else:
      model.eval() #set model to eval mode


    #keep a higher level list of running metric values
    rwer = 0
    epoch_agg_metrics = {}

    for idx, batch in enumerate(tqdm.tqdm(data)):

      #get the specific pieces of information we need from each sample
      input_ids = batch.pop("input_ids").to(device)
      pixel_values = batch.pop("pixel_values").to(device)
      attention_mask = batch.pop("attention_mask").to(device)

      optimizer.zero_grad() #zero out the gradient

      with torch.set_grad_enabled(name == 'train'):

        #send the samples to the model
        outputs = model(input_ids=input_ids,
                        pixel_values=pixel_values,
                        labels=input_ids,
                        attention_mask=attention_mask)

        #get and print loss
        loss = outputs.loss
        print(f"{name} loss:", loss.item())

        #backprop
        if name == 'train':
          loss.backward()
          optimizer.step()

      #check how the current model at this stage of training fares on wer metrics
      generated_ids = model.generate(pixel_values=pixel_values, max_new_tokens=500) #received warnings about degraded performance when I set max_new_tokens beyond 500, so keeping 500 here.
      generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)
      gt_caption = processor.batch_decode([i[:500] for i in input_ids], skip_special_tokens=True) #truncating tokens to 500 so model is not unfairly penalized for 'deleted'/missing words when/if ground truth tokens exceed 500

      if idx % 100 == 0:
        print(f"This was the generated caption: {generated_caption}")
        print(f"This was the true caption: {gt_caption}")

      wer_score = wer.compute(predictions=generated_caption, references=gt_caption)
      rwer += wer_score

      #calculate the agg metrics for every 100 step interval. Logging this every 100 steps for the same reason listed below.
      if idx % 100 == 0:
        agg_wer = rwer/(idx+1)
        print(f"Step {idx+1} -- wer: {agg_wer}")


      #additional, more frequent checkpointing in case colab sessions time out
      if idx%1000 == 0:
        if agg_wer < best_wer:
          best_wer = agg_wer
          best_model_wts_wer = copy.deepcopy(model.state_dict())
          torch.save(best_model_wts_wer, f'drive/MyDrive/fine_tuned_blip_wer_{name}_idx_{idx}')

      #keep track of how well the model is doing on wer
      #(note: the training dataset is massive and BLIP is relatively sensitive/response to samples, in my opinion (i.e., within a couple hundred
      #steps, we can see pretty noticeable changes in training loss, wer scores, and outputted text). So I'm logging the wer metrics
      #and saving the weights every 1000 steps instead of every epoch.)
      if name == 'val' and idx%1000 == 0:
        if agg_wer < best_wer: #the lower the score, the better
          best_wer = agg_wer
          best_model_wts_wer = copy.deepcopy(model.state_dict())
          torch.save(best_model_wts_wer, 'drive/MyDrive/fine_tuned_blip_wer_WEIGHTS_ONLY')

print(f"BEST WER: {best_wer}")

Epoch: 0


  0%|          | 0/16165 [00:00<?, ?it/s]

train loss: 6.785060882568359


  0%|          | 1/16165 [00:01<8:13:39,  1.83s/it]

This was the generated caption: ['there are many birds sitting on a wire with music notes', 'arafed wire with birds sitting on it']
This was the true caption: ['* * ekern _ 1. 0 < t > * * ekern _ 1. 0 < b > * cleff4 < t > * clefg2 < b > * k [ b - ] < t > * k [ b - ] < b > * m3 / 4 < t > * m3 / 4 < b > = - < t > = - < b > 4cc < t > 8. ddl < b >. < t > 16ccjk < b > 4c < s > 4e < s > 4b - < t > 4b - < b > 4c < s > 4e < s > 4b - < t > 8al < b >. < t > 8gj < b > = < t > = < b > 4ff < t > 8fl < b >. < t > 8gj < b > 4c < s > 4f < s > 4a < t > 4a < b > 4c < s > 4f < s > 4a < t > 8al < b >. < t > 8b - j < b > = < t > = < b > 4cc < t > 8ddl < b >. < t > 16r < b >. < t > 16ccjk < b > 4c < s > 4e < s > 4b - < t > 4b - < b > 4c < s > 4e < s > 4b - < t > 8al < b >. < t > 8gj < b > = < t > = < b > * ^ < t > * < b > 4ff < < t > 4r < t > 8fl < b >. < t >. < t > 8gj < b > 4f < s > 4a < t > 8cl < t > 2a < b >. < t > 8c # j < t >. < b > 4r < t > 4d < s > 4f < s > 4a < t >. < b > * v < t > * v < t > * < b 

...

train loss: 1.064387559890747


  2%|▏         | 278/16165 [42:41<50:50:43, 11.52s/it]

train loss: 0.6689918041229248


  2%|▏         | 279/16165 [42:53<50:50:30, 11.52s/it]

train loss: 1.0510791540145874


In [ ]:
#save the model in case the training was cancelled/"keyboard-interrupted" early bc of session limits 
best_model_wts = copy.deepcopy(model.state_dict())
torch.save(best_model_wts, 'drive/MyDrive/fine_tuned_blip_early_stop_WEIGHTS_ONLY') #change path as needed

### Code Comments

Training loop looks good and its awesome to see the progression from traditional language model to bekrn format
Makes me wonder what the outputs look like between the first and second printed output and what an output would look like for a non sheetmusic image input in the finetuned model.

Naturally things look cleaner broken into functions, but not worth it if there is only one dataset that we plan to use. 

In [ ]:
#for loading in a saved set of model weights when needed
model.load_state_dict(torch.load("drive/MyDrive/fine_tuned_blip_wer_val_idx_0")) #change path as needed

### Evaluation of Fine-Tuned BLIP

In [ ]:
#let's now see how the fine-tuned BLIP model does with the test set. Like before, eval here will be based on Word Error Rate (WER) which 
#seems to be a better metric for this use case than other popular metrics like rouge-n. WER approximates the label of "incorrect" words divided by 
#the total number of reference words. WER=1.0 means a perfect mismatch while WER=0.0 indicates a perfect match. So the lower the score, the better.

#(WER is sensitive to whether or not predicted words are exact matches to the reference labels + the order in which predicted words appear, two properties
#that are very important for our use case. E.g., if a predicted sequence has all the same words as the reference sequence but in a different order -- that is 
#effectively a completely different musical score. 

#The original paper that produced the raw dataset we're using also uses variations of WER to judge baseline model runs on its dataset.)

### Comments

Markdown is good

In [ ]:
model.eval() # set the model to eval mode if it wasn't there already

all_gt_ids, all_gt_text = [], []
all_pred_ids, all_pred_text = [], []

rwer = 0

with torch.no_grad(): #not computing gradients while testing
  for idx, batch in enumerate(tqdm.tqdm(test_dataloader_blip)):

    #retrieve the image and gt labels
    pixel_values = batch.pop("pixel_values").to(device)
    labels = batch.pop("input_ids").to(device)

    #make prediction and get both the outputted ids and the text caption
    generated_ids = model.generate(pixel_values=pixel_values, max_new_tokens=500)
    generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)

    #get the text of the labels
    gt_caption = processor.batch_decode([i[:500] for i in labels], skip_special_tokens=True)

    wer_score = wer.compute(predictions=generated_caption, references=gt_caption)
    rwer += wer_score

    if idx % 100 == 0:
      agg_wer = rwer/(idx+1)
      print(f"Step {idx+1} -- wer: {agg_wer}")

    #store the preds and gts for eval later
    for pred_ids, pred_cap, gt_ids, gt_cap in zip(generated_ids, generated_caption, labels, gt_caption):
      all_pred_ids.append(pred_ids)
      all_pred_text.append(pred_cap)
      all_gt_ids.append(gt_ids)
      all_gt_text.append(gt_cap)

In [ ]:
#saving the predictions and the gt labels bc a full test run takes over a day w/ the gpu i'm on.
#(so if more metrics need to be calc'd later or predictions need to be viewed, we can just load this back in)
with open('drive/MyDrive/all_pred_ids.pickle', 'wb') as output:
    pickle.dump(all_pred_ids, output)
with open('drive/MyDrive/all_pred_text.pickle', 'wb') as output:
    pickle.dump(all_pred_text, output)
with open('drive/MyDrive/all_gt_ids.pickle', 'wb') as output:
    pickle.dump(all_gt_ids, output)
with open('drive/MyDrive/all_gt_text.pickle', 'wb') as output:
    pickle.dump(all_gt_text, output)

### Code Comments

Overall, everything looks great. There are of course some code blocks that could have been functions, but that's kind of the nature of Jupyter notebooks. Not too much to say other than cleaning up some extraneous print statements in the preprocessing section and maybe consider moving some of your longer comments to markdown blocks or docstrings. There are a couple places you use hardcoded variables despite getting values for them previously in the notebook. You may also want to 